# Qiskit's Particle Hole Transformer
### Author: Justice M. Calderón

In this document, I summarize some notes and thoughts on the Particle Hole transformation implemented in Qiskit. The origin of these notes comes from a variety of discussions which came up in the process of migrating the implementation to Qiskit Nature.

## Implementation State in Qiskit Aqua

The implementation in Qiskit Aqua was split in two parts:
- [`FermionicOperator.particle_hole_transformation`](https://github.com/Qiskit/qiskit-aqua/blob/7229dd15b01e5015ec649a3264561fc01d39a16b/qiskit/chemistry/fermionic_operator.py#L501)
- [`particle_hole_transformation`](https://github.com/Qiskit/qiskit-aqua/blob/master/qiskit/chemistry/particle_hole.py)

## Implementation in Qiskit Terra (Qiskit Aqua is deprecated)

The first part dealt with transforming the integrals stored in the `FermionicOperator` between block- and interleaved-spin format whereas the second part dealt with the actual transformation.

Below is a simple example showcasing how the transformation works.

In [1]:
# Importing relevant packages

from qiskit_nature.second_q.operators import FermionicOp, ElectronicIntegrals, PolynomialTensor
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import *
from qiskit_nature.second_q.algorithms import NumPyMinimumEigensolverFactory

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
#driver = PySCFDriver()
#q_mol = driver.run()
#fer_op = FermionicOp(h1=q_mol.one_body_integrals, h2=q_mol.two_body_integrals)

In [4]:
# First, let's check what the expected result is:
#qubit_op = fer_op.mapping('jordan_wigner')
#result = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
#print('E_elec = {result.eigenvalue.real}')

Next, we need to return the one- and two-body integrals. For reference, we need to reference the ElectronicEnergy and ElectronicIntegrals classes in Qiskit

ref [1] https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.hamiltonians.ElectronicEnergy.html#qiskit_nature.second_q.hamiltonians.ElectronicEnergy)

ref [2] https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.operators.ElectronicIntegrals.html#qiskit_nature.second_q.operators.ElectronicIntegrals)

In [20]:
# Setting up the electronic structure problem with the Jordan-Wigner mapping
driver = PySCFDriver()
q_mol = driver.run()
fer_op = FermionicOp(q_mol.hamiltonian.second_q_op()) # Returning the FermionicOp() of the driver results

In [36]:
# Converting the fermionic operators to qubit operators using the Jordan-Wigner mapping
qubit_op = JordanWignerMapper().map(fer_op)
min_eig_solver = NumPyMinimumEigensolverFactory().minimum_eigensolver
result = min_eig_solver.compute_minimum_eigenvalue(qubit_op)
print("E_elec = {}".format(result.eigenvalue.real))

E_elec = -1.8572750302023804


As we can see, the qubit operators are in fact identical except for the unity-term, `IIII`. This term is shifted by `ph_energy_shift`.

Technically this means, that we can achieve the desired result of the p/h transformer, by only including a single extra term, which shifts the energy of the unity Pauli string. However, this does not reflect the mathematical implementation of the actual flow at all. Therefore, I strongly discourage this approach, especially because we do not fully understand all of the underlying assumptions introduced when doing this.

## Moving to Qiskit Nature

Qiskit Nature provides us the `FermionicOp` which is an operator that properly implements the fermionic creation (`+`) and annihilation (`-`) operations. In doing so, it also implements the correct anti-commutation rules and provides the means to normal order the operators.

This means, that we can envision an implementation of the p/h transformer which leverages the proper anti-commutation built into the `FermionicOp`. To this extent, I shortly demonstrate the normal-ordering of the `FermionicOp`:

For reference, see the following links to understand normal ordering as used in quantum field theory

https://en.wikipedia.org/wiki/Normal_order#Fermions

https://www.youtube.com/watch?v=bv77qlOkIEI

https://qiskit.org/documentation/nature/_modules/qiskit_nature/second_q/operators/fermionic_op.html#FermionicOp.normal_order

https://en.wikipedia.org/wiki/Normal_order#Multiple_fermions

In [9]:
import numpy as np
from itertools import permutations, product
from functools import reduce
from qiskit_nature.operators.second_quantization import FermionicOp

In [10]:
# Let's check the normal-ordering of the FermionicOp

# We iterate all possible double-excitation index combinations
for ijkl in product(*[range(4)] * 4):
    if ijkl[0] > 0:
        # for readability we only print the ones where i == 0
        break
        
    # We then iterate the possible permutations of creation and annihilations operators
    for operators in set(permutations('++--', 4)):
        # the integrals which we deal with are in blocked spin format. Thus, the following association holds
        #   i -> alpha-spin occupied
        #   j -> alpha-spin unoccupied
        #   k -> beta-spin occupied
        #   l -> beta-spin unoccupied
        # For chemistry applications, we apply them in normal order to get the anti-commutation for free:
        iklj, operators_reordered = zip(*np.asarray(list(zip(ijkl, operators)))[[0, 2, 3, 1]])
        
        # We construct the operators via a reduction because this allows us to compose sparse-label based ops
        # in any order (regardless of index order limitations in the parser)
        op = reduce(lambda a, b: a @ b, [FermionicOp(f'{op}_{idx}', register_length=4)
                                         for idx, op in zip(iklj, operators_reordered)])
        op = op.reduce()
        if op._coeffs != [0j]:
            # if the reduced operator is not empty, we compute the normal ordered operator
            normal_ordered_op = op.to_normal_order_op().reduce()
            # ... and then print the result
            print(ijkl, operators, op, str(normal_ordered_op).replace('\n', ' '))

(0, 0, 0, 0) ('-', '+', '+', '-') IIIE * (1+0j)   IIII * (1+0j) + IIIN * (-1+0j)
(0, 0, 0, 0) ('+', '-', '-', '+') IIIN * (1+0j) IIIN * (1+0j)
(0, 0, 0, 1) ('-', '-', '+', '+') II+- * (1+0j) II+- * (1+0j)
(0, 0, 0, 1) ('+', '+', '-', '-') II-+ * (1+0j) II-+ * (1+0j)
(0, 0, 0, 2) ('-', '-', '+', '+') I+I- * (1+0j) I+I- * (1+0j)
(0, 0, 0, 2) ('+', '+', '-', '-') I-I+ * (1+0j) I-I+ * (1+0j)
(0, 0, 0, 3) ('-', '-', '+', '+') +II- * (1+0j) +II- * (1+0j)
(0, 0, 0, 3) ('+', '+', '-', '-') -II+ * (1+0j) -II+ * (1+0j)
(0, 0, 1, 0) ('-', '-', '+', '+') II+- * (-1+0j) II+- * (-1+0j)
(0, 0, 1, 0) ('+', '+', '-', '-') II-+ * (-1+0j) II-+ * (-1+0j)
(0, 0, 1, 1) ('+', '-', '+', '-') IINN * (1+0j) IINN * (1+0j)
(0, 0, 1, 1) ('-', '+', '+', '-') IINE * (1+0j)   IINI * (1+0j) + IINN * (-1+0j)
(0, 0, 1, 1) ('+', '-', '-', '+') IIEN * (1+0j)   IIIN * (1+0j) + IINN * (-1+0j)
(0, 0, 1, 1) ('-', '+', '-', '+') IIEE * (1+0j)   IIII * (1+0j) + IIIN * (-1+0j) + IINI * (-1+0j) + IINN * (1+0j)
(0, 0, 1, 2) ('+', 

(0, 3, 3, 1) ('+', '-', '+', '-') NI-+ * (1+0j) NI-+ * (1+0j)
(0, 3, 3, 1) ('+', '+', '-', '-') EI-+ * (1+0j)   II-+ * (1+0j) + NI-+ * (-1+0j)
(0, 3, 3, 1) ('-', '+', '-', '+') EI+- * (1+0j)   II+- * (1+0j) + NI+- * (-1+0j)
(0, 3, 3, 2) ('-', '-', '+', '+') N+I- * (1+0j) N+I- * (1+0j)
(0, 3, 3, 2) ('+', '-', '+', '-') N-I+ * (1+0j) N-I+ * (1+0j)
(0, 3, 3, 2) ('+', '+', '-', '-') E-I+ * (1+0j)   I-I+ * (1+0j) + N-I+ * (-1+0j)
(0, 3, 3, 2) ('-', '+', '-', '+') E+I- * (1+0j)   I+I- * (1+0j) + N+I- * (-1+0j)
(0, 3, 3, 3) ('-', '+', '+', '-') +II- * (-1+0j) +II- * (-1+0j)
(0, 3, 3, 3) ('+', '-', '-', '+') -II+ * (-1+0j) -II+ * (-1+0j)


We can see, that the normal-ordered operators correctly reproduce the anti-commutation rules which are currently hard-coded in Aqua's p/h implementation. **However**, we also see, that some important relations are missing, because the corresponding `FermionicOp` are zero-operators. For example:
$NO[\hat{a}_0 \hat{a}_0^\dagger \hat{a}_0 \hat{a}_1^\dagger] = - \hat{a}_0^\dagger \hat{a}_1^\dagger \hat{a}_0 \hat{a}_0 - \hat{a}_1^\dagger \hat{a}_0$, but Qiskit produces:

In [11]:
ijkl, operators = (0, 0, 1, 0), ('-', '+', '-', '+')
iklj, operators_reordered = zip(*np.asarray(list(zip(ijkl, operators)))[[0, 2, 3, 1]])
op = reduce(lambda a, b: a @ b, [FermionicOp(f'{op}_{idx}', register_length=4)
                                 for idx, op in zip(iklj, operators_reordered)])
print(op)

IIII * 0j


The reason for this limitation is the double $\hat{a}_0$ operator, which is an invalid operation on a two-level system. Therefore, the current implementation may not suffice for the p/h needs yet. However, since we want to extend the `FermionicOp` in the future, to use a sparse, operator-order preserving internal data structure (before `reduce()` at least), we should be able to support this case, too.

Before finishing this discussion, I would quickly like to note down two other requirements, which would enable us to implement the p/h transformer as a `QMolecule -> QMolecule` transformer (i.e. a subclass of `qiskit_nature.transformers.base_transformer.BaseTransformer`):
  - [ ] We need to be able to handle shifting values between the `h2_aa`, `h2_ba` and `h2_bb` matrices:
    - the reason for this, is that the re-interpretation and re-ordering of the labels during the p/h transformation can redistribute the `h2` integrals values
  - [x] We need to be able to compute the p/h energy shift directly in the MO basis:
    - in fact, this is already possible leveraging the `ActiveSpaceTransformer` with an empty active space: `num_electrons=0` and `num_orbitals=0`

In the process of writing these notes and debugging the old p/h implementation, a few open questions turned op:
  - [ ] Why do the `h1` integrals **not** change in the old code? Surely, :math:`<r|F|s>` differs from :math:`<r|h|s>`...
  - [ ] Would we even want to mix a `QMolecule -> QMolecule` transformer with using the `FermionicOp` code?
    - (this is more of a question for the future implementation)
  - [ ] And finally: there is a bug in the old p/h implementation in Aqua, which changes the integrals of the `FermionicOperator` being transformed. See the example below.

In [12]:
driver = PySCFDriver()
q_mol = driver.run()
fer_op = FermionicOperator(h1=q_mol.one_body_integrals, h2=q_mol.two_body_integrals)
print(fer_op.h1)
ph_fer_op, _ = fer_op.particle_hole_transformation((q_mol.num_alpha, q_mol.num_beta))
print(fer_op.h1)

[[-1.25633907  0.          0.          0.        ]
 [ 0.         -0.47189601  0.          0.        ]
 [ 0.          0.         -1.25633907  0.        ]
 [ 0.          0.          0.         -0.47189601]]
[[-1.25633907  0.          0.          0.        ]
 [ 0.         -1.25633907  0.          0.        ]
 [ 0.          0.         -0.47189601  0.        ]
 [ 0.          0.          0.         -0.47189601]]


Notice how the same object has been converted to interleaved-spins although this should be reverted back to blocked-spins.